In [ ]:
#tensorboard via command-line
#tensorboard --logdir=/full_path_to_your_logs

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
#What is the maximum flare class produced by an AR in the next 24hrs after a 24hr time sequence?
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
#feature data from - hmi.sharp_720s:::
#http://jsoc.stanford.edu/doc/data/hmi/sharp/sharp.htm

#label data from - GOES flare events:::
#ftp://ftp.swpc.noaa.gov/pub/warehouse/

#and from - GOES XRS Report:::
#https://www.ngdc.noaa.gov/stp/space-weather/solar-data/solar-features/solar-flares/x-rays/goes/xrs/
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
#binary class - F for flare and N for no flare
#multi-class - #6 flare classes: N is no flare, A is smallest, X is largest
               #key = {'N', 'A', 'B', 'C', 'M', 'X'}
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

In [ ]:
import tensorflow as tf
print("tensorflow version:", tf.__version__) #2.3.1 used
from tensorflow import keras
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import datetime
import shutil
import glob
import os
from astropy.time import Time
from pathlib import Path
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dropout, Dense, Embedding, Flatten, SimpleRNN, LSTM, GRU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import SGD, Adam
import drms #https://pypi.org/project/drms/

In [ ]:
import Flare_Data

In [ ]:
################creates/loads csv with 2012's feature data##########
current_di = os.getcwd()
csv_for_2012 = "\\create_2012_features.csv"
where_2012 = current_di + csv_for_2012

if not os.path.exists(where_2012):
    import to_create_2012
    to_create_2012.get_2012_Features()
    df_2012 = pd.read_csv('create_2012_features.csv', index_col=0)
    df_2012_done = Flare_Data.convert_time_2012(df_2012)
    Flare_Data.save_some_features(df_2012_done) 

if os.path.exists(where_2012):
    df_2012 = pd.read_csv('create_2012_features.csv', index_col=0)
    df_2012_done = Flare_Data.convert_time_2012(df_2012)
    Flare_Data.save_some_features(df_2012_done)
####################################################################

In [ ]:
################creates/loads csv with 2013's feature data##########
current_di = os.getcwd()
csv_for_2013 = "\\create_2013_features.csv"
where_2013 = current_di + csv_for_2013

if not os.path.exists(where_2013):
    import to_create_2013
    to_create_2013.get_2013_Features()
    df_2013 = pd.read_csv('create_2013_features.csv', index_col=0)
    df_2013_done = Flare_Data.convert_time_2013(df_2013)
    Flare_Data.save_some_features(df_2013_done) 

if os.path.exists(where_2013):
    df_2013 = pd.read_csv('create_2013_features.csv', index_col=0)
    df_2013_done = Flare_Data.convert_time_2013(df_2013)
    Flare_Data.save_some_features(df_2013_done)
####################################################################

In [ ]:
################creates/loads csv with 2014's feature data##########
current_di = os.getcwd()
csv_for_2014 = "\\create_2014_features.csv"
where_2014 = current_di + csv_for_2014

if not os.path.exists(where_2014):
    import to_create_2014
    to_create_2014.get_2014_Features()
    df_2014 = pd.read_csv('create_2014_features.csv', index_col=0)
    df_2014_done = Flare_Data.convert_time_2014(df_2014)
    Flare_Data.save_some_features(df_2014_done) 

if os.path.exists(where_2014):
    df_2014 = pd.read_csv('create_2014_features.csv', index_col=0)
    df_2014_done = Flare_Data.convert_time_2014(df_2014)
    Flare_Data.save_some_features(df_2014_done)
####################################################################

In [ ]:
################creates/loads csv with 2015's feature data##########
current_di = os.getcwd()
csv_for_2015 = "\\create_2015_features.csv"
where_2015 = current_di + csv_for_2015

if not os.path.exists(where_2015):
    import to_create_2015
    to_create_2015.get_2015_Features()
    df_2015 = pd.read_csv('create_2015_features.csv', index_col=0)
    df_2015_done = Flare_Data.convert_time_2015(df_2015)
    Flare_Data.save_some_features(df_2015_done) 

if os.path.exists(where_2015):
    df_2015 = pd.read_csv('create_2015_features.csv', index_col=0)
    df_2015_done = Flare_Data.convert_time_2015(df_2015)
    Flare_Data.save_some_features(df_2015_done)
####################################################################

In [ ]:
#get_ipython().run_line_magic('load_ext', 'tensorboard')
#%load_ext tensorboard

In [ ]:
#removes old tensorboard Logs and recreates or creates Logs folder
current_dir = os.getcwd()
l_folder = "\\Logs"
where_logs = current_dir + l_folder

if not os.path.exists(where_logs):
    os.mkdir(where_logs)
    
if os.path.exists(where_logs):
    shutil.rmtree(where_logs)
    os.mkdir(where_logs)

In [ ]:
#binary = True for binary classification, binary = False for multi-class classification
X_train, X_val, X_test, y_train, y_val, y_test, tim_steps, n_feats, count_of_classes = Flare_Data.getAllData(binary = False)

In [ ]:
print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

In [ ]:
batch_size = 1
epochs = 10

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~[LSTM_1 Model]~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(tim_steps, n_feats)))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(count_of_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

In [ ]:
tb_cb = keras.callbacks.TensorBoard(
    log_dir=where_logs,
    histogram_freq=0,  # how often to log histogram visualizations
    embeddings_freq=1,  # how often to log embedding visualizations
    update_freq="epoch",
)

In [ ]:
history = model.fit(X_train, y_train, 
          batch_size=batch_size, 
          epochs=epochs, 
          validation_data=(X_val, y_val), 
          callbacks=[tb_cb])

In [ ]:
history.history

In [ ]:
#evaluate model on test set
results = model.evaluate(X_test, y_test, batch_size=1)
print("test loss, test acc:", results)

In [ ]:
#generate predictions for 3
predictions = model.predict(X_test[:3])
print("predictions:", predictions)

In [ ]:
#confusion matrix
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test.argmax(axis=1), predicted_flare_class.argmax(axis=1))
print(matrix)